# Sensitivity/Specificity analysis demo

First, we configure several variables we are using during this demo: project id, S3 profile and VM path to save data files from the S3 bucket: 

In [ ]:
import json
import requests
import warnings
warnings.filterwarnings('ignore')

project = 'bpa-ThermoFisher_P0001_T1'
profile = 'bloodpac'
path    = 'files/'

### 1. Querying data from graphQL

These are some examples of queries to directly interact with the API to get some metadata:

In [ ]:
credentials = 'credentials.json'    
json_data=open(credentials).read()
keys = json.loads(json_data)
auth = requests.post('https://data.bloodpac.org/user/credentials/cdis/access_token', json=keys)   

**A) Query general API to get projects included in BloodPAC:**

In [ ]:
print requests.get('https://data.bloodpac.org/api/v0/submission/bpa/', 
                   headers={'Authorization': 'bearer '+ auth.json()['access_token']}).text

**B) Example graphQL query:** List submitted aligned read files (BAMs) for a specific project:

In [ ]:
data = {'query': """{sample (project_id: "bpa-ThermoFisher_P0001_T1") {   
                                 _aliquots_count 
                                 aliquots {  
                                     analytes{
                                         _read_groups_count 
                                         read_groups {
                                            _submitted_aligned_reads_files_count
                                            submitted_aligned_reads_files {
                                                file_name
                                            }
                                         } 
                                     }
                                }
                            }
                    } """};

print requests.post('https://data.bloodpac.org/api/v0/submission/graphql/', 
                    headers={'Authorization': 'bearer '+ auth.json()['access_token']}, 
                    json=data).text

### 2. Exploring metadata through a high level Python library

Import the Python library with functionality to explore metadata and make some data analyses. Also, add our authentication file to the library. 

In [ ]:
import bpa_analysis_functions_v2 as bp
bp.add_keys('credentials.json')

First, we show some functions to explore information in the project's metadata: 

**A) List samples for this project:**

In [ ]:
samples = bp.list_samples(project)
bp.arrayTable(samples)

**B) List files in one project by type:** 

In [ ]:
vcf_files = bp.list_files_by_type(project,'VCF')
bp.SummaryTable(vcf_files)

**C) Get summary metrics for one field in a specific node:**

Get expected mutations from contrived_expectation metadata and primary diagnosis:

In [ ]:
summary = bp.query_summary_field("contrived_expectation", "expected_mutation_gene")
summary = bp.query_summary_field("diagnosis", "primary_diagnosis")

### 3. Data handle and analysis

Now, we use additional functions to show how we can analyze data from one specific project in BloodPAC containing expected mutations from contrived samples and resulting VCF files. 

**A) Transfer files from S3 bucket to the VPC virtual machine:**

In [ ]:
bp.get_files_from_bucket(project, profile, path, '*.vcf')

**B) Calculate quality metrics (sensitivity/sensitivity)**:

In [ ]:
sample = samples[1]
vcf_file = vcf_files[sample][8]
metrics = bp.calculate_metrics_vcf(project, path, vcf_file)
metrics

Calculate sensitivity and sensitivity values for all VCFs in the project:

In [ ]:
table_metrics, data_metrics = bp.calculate_metrics_all_vcf(project, path, vcf_files, samples[0])
table_metrics

Remove potential germline mutations using a baseline VCF:

In [ ]:
baseline_vcf = 'TFS.HZ.0.0.perc_20ng_TSVC_IonCodeTag_0101.vcf'
table_filter_metrics, data_filter_metrics = bp.calculate_metrics_all_vcf(project, path, vcf_files, samples[0], baseline_vcf)
table_filter_metrics

Some results are finally visualized in barplots:

In [ ]:
bp.plot_metrics(data_metrics, data_filter_metrics)